# Import packages

In [624]:
import pandas as pd

In [625]:
import plotly
import datapane as dp
plotly.offline.init_notebook_mode(connected=True)

In [626]:
import re
import unicodedata

import pickle

import Levenshtein
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Upload json files

In [627]:
#files= ['0.json', '1.json', '2.json', '3.json', '4.json','42.json', '15693.json']
files = ['0.json']
# create an empty list to store the DataFrames
dfsList = [pd.read_json(file, orient='records') for file in files]

# combine all DataFrames into a single DataFrame
crossrefDF = pd.concat(dfsList, ignore_index=True)


In [628]:
crossrefDF

,items
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...
2,{'abstract': '<jats:p> <jats:bold>...
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...
4,{'abstract': '<jats:p>The integration of the I...
...,...
4995,{'URL': 'http://dx.doi.org/10.1053/tvjl.2002.0...
4996,{'abstract': '<jats:p>This article reports on ...
4997,{'abstract': '<jats:p> We present a collection...
4998,{'abstract': '<jats:p>Looks at the nature of w...


# Data preparation 

In [629]:
noAuthors = [i for i in range(len(crossrefDF)) if 'author' not in crossrefDF['items'][i]]

Authors = [i for i in range(len(crossrefDF)) if 'author'  in crossrefDF['items'][i]]

In [630]:
len(noAuthors) + len(Authors) == len(crossrefDF)

True

## Rows with authors

In [631]:
crossrefAuth = crossrefDF.iloc[Authors].copy()

crossrefAuth.reset_index(inplace= True)
crossrefAuth.drop(columns = ['index'], inplace = True)


## Extract 'DOI'

In [632]:
crossrefAuth.loc[:, 'DOI'] = crossrefAuth['items'].apply(lambda x: x['DOI'])

In [633]:
crossrefAuth.head()

,items,DOI
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338


## Extract 'authors' --- number of authors

In [634]:
crossrefAuth.loc[:,'authors'] = crossrefAuth['items'].apply(lambda x: x['author'])


In [635]:
crossrefAuth.head()

,items,DOI,authors
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque..."
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ..."
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se..."
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'..."
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq..."


In [636]:
numAuthors = [len(crossrefAuth.iloc[i]['authors']) for i in range(len(crossrefAuth))]

In [637]:
## yparxoun lathi  ---> kalytera number of affiliations
crossrefAuth.loc[:,'# authors'] = numAuthors

In [638]:
crossrefAuth.head()

,items,DOI,authors,# authors
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ...",3
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'...",1
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq...",2


## Extract 'affiliations' --- number of affiliations

In [639]:
def getAff(k):
   return [crossrefAuth['authors'][k][j]['affiliation'] for j in range(len(crossrefAuth['authors'][k]))]
    

In [640]:
Affiliations = [getAff(k) for k in range(len(crossrefAuth))]

crossrefAuth.loc[:,'affiliations'] = Affiliations


In [641]:
numAffil = [len(Affiliations[i]) for i in range(len(crossrefAuth))]

In [642]:
crossrefAuth.loc[:,'# Affil'] = numAffil

In [643]:
crossrefAuth.head()

,items,DOI,authors,# authors,affiliations,# Affil
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,{'URL': 'http://dx.doi.org/10.1103/physrevc.65...,10.1103/physrevc.65.064904,"[{'given': 'S. V.', 'family': 'Akkelin', 'sequ...",3,"[[], [], []]",3
2,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
3,{'URL': 'http://dx.doi.org/10.1093/jjco/hyf035...,10.1093/jjco/hyf035,"[{'given': 'E.', 'family': 'Bakan', 'sequence'...",1,[[]],1
4,{'abstract': '<jats:p>The integration of the I...,10.1108/14684520010357338,"[{'given': 'Kaba', 'family': 'Abdoulaye', 'seq...",2,"[[], []]",2


## Clean 'empty' affiliations

In [644]:
possibleEmptyAff = []

for k in range(len(crossrefAuth)):
    if len(crossrefAuth['affiliations'][k][0]) == 0:
        possibleEmptyAff.append(k)

In [645]:
len(possibleEmptyAff)

3767

In [646]:
nonEmptyAff = []

for k in possibleEmptyAff:
    for j in range(len(crossrefAuth['affiliations'].iloc[k])):
        if len(crossrefAuth['affiliations'].iloc[k][j]) != 0:
            nonEmptyAff.append(k)
    
    

In [647]:

FinalEmptyyAff=  [x for x in possibleEmptyAff if x not in nonEmptyAff] 

In [648]:
FinalNonEmptyAff = [x for x in range(len(crossrefAuth)) if x not in FinalEmptyyAff]

# affilDF: crossrefAuth subdataframe with nonpempty affiliation lists

In [649]:
affilDF = crossrefAuth.iloc[FinalNonEmptyAff].copy()
affilDF.reset_index(inplace = True)
affilDF.drop(columns = ['index'], inplace = True)

## (still some cleaning: cases with empty brackets [{}])

In [650]:
affilDF[affilDF['DOI'] == '10.48130/emst-2022-0020']

,items,DOI,authors,# authors,affiliations,# Affil


In [651]:
for k in range(len(affilDF)):
    if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}:
        print(k)

In [652]:
emptyBrackets = [k for k in range(len(affilDF)) if len(affilDF['affiliations'][k][0]) != 0 and affilDF['affiliations'][k][0][0] == {}]

In [653]:
affilDF.iloc[emptyBrackets]

,items,DOI,authors,# authors,affiliations,# Affil


In [654]:
affilDF.copy()

,items,DOI,authors,# authors,affiliations,# Affil
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3
3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6
4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5
...,...,...,...,...,...,...
882,"{'URL': 'http://dx.doi.org/10.1021/ic0255961',...",10.1021/ic0255961,"[{'given': 'Fatima', 'family': 'Setifi', 'sequ...",11,[[{'name': 'Laboratoire de Chimie du Solide et...,11
883,"{'URL': 'http://dx.doi.org/10.1021/jf011259j',...",10.1021/jf011259j,"[{'given': 'Alexandre G.', 'family': 'Torres',...",3,[[{'name': 'Laboratório de Bioquímica Nutricio...,3
884,"{'URL': 'http://dx.doi.org/10.1021/jp014423e',...",10.1021/jp014423e,"[{'given': 'Keisaku', 'family': 'Kimura', 'seq...",3,"[[{'name': 'Department of Material Science, Hi...",3
885,{'abstract': '<jats:p> The actin filament-asso...,10.1091/mbc.e01-12-0148,"[{'given': 'Yong', 'family': 'Qian', 'sequence...",12,[[{'name': 'The Mary Babb Randolph Cancer Cent...,12


In [655]:
affilDF.drop(emptyBrackets, inplace = True)

In [656]:
affilDF.reset_index(inplace = True)

In [657]:
affilDF.copy()

,index,items,DOI,authors,# authors,affiliations,# Affil
0,0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3
1,1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10
2,2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3
3,3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6
4,4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5
...,...,...,...,...,...,...,...
882,882,"{'URL': 'http://dx.doi.org/10.1021/ic0255961',...",10.1021/ic0255961,"[{'given': 'Fatima', 'family': 'Setifi', 'sequ...",11,[[{'name': 'Laboratoire de Chimie du Solide et...,11
883,883,"{'URL': 'http://dx.doi.org/10.1021/jf011259j',...",10.1021/jf011259j,"[{'given': 'Alexandre G.', 'family': 'Torres',...",3,[[{'name': 'Laboratório de Bioquímica Nutricio...,3
884,884,"{'URL': 'http://dx.doi.org/10.1021/jp014423e',...",10.1021/jp014423e,"[{'given': 'Keisaku', 'family': 'Kimura', 'seq...",3,"[[{'name': 'Department of Material Science, Hi...",3
885,885,{'abstract': '<jats:p> The actin filament-asso...,10.1091/mbc.e01-12-0148,"[{'given': 'Yong', 'family': 'Qian', 'sequence...",12,[[{'name': 'The Mary Babb Randolph Cancer Cent...,12


In [658]:
affilDF.drop(columns = ['index'], inplace = True)

# Clean affiliations 

## is_contained(a,b) map : returns true when a is a substring of b 

In [659]:
def is_contained(s, w):
    words = s.split()  # Split the string 's' into a list of words
    for word in words:
        if word not in w:  # If a word from 's' is not found in 'w'
            return False  # Return False immediately
    return True  # If all words from 's' are found in 'w', return True

## 1. "Unique" affiliations --- number of unique affiliations

In [660]:
uniqueAff = []
error_indices =[] # New list to store error indices
for i in range(len(affilDF)):
    try:
        uniqueAff.append(list(set([x[0] for x in [list(d.values()) for d in [item for sublist in affilDF['affiliations'].iloc[i] for item in sublist if sublist !=[{}]]]])))
    except TypeError:
        print("Error occurred for i =", i)
        error_indices.append(i)  # Save the index where the error occurred
    #except IndexError:
     #   print("IndexError occurred for i =", i)
      #  error_indices.append(i)  # Save the index where the IndexError occurred


# Print the error indices
print("Error indices:", error_indices)

Error indices: []


In [661]:
affilDF.drop(error_indices, inplace = True)
affilDF.reset_index(inplace = True)
affilDF.drop(columns = ['index'], inplace = True)

In [662]:
affilDF.loc[:,'uniqueAff'] = uniqueAff

In [663]:
numUniqueAff = [len(affilDF['uniqueAff'].iloc[i]) for i in range(len(affilDF))]

In [664]:
affilDF.loc[:,'# uniqueAff'] = numUniqueAff

## 2. Remove stop words ['from', 'the']

In [665]:
stopWords = ['from', 'the', 'From', 'The', 'of', 'at', 'de']

In [666]:
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopWords]
    return ' '.join(filtered_words)


# apply the function to the column  affilDF['uniqueAff'] to create column affilDF.loc[:,'uniqueAff1']

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff'].apply(lambda x: [remove_stop_words(s) for s in x])


## 3. Remove parenthesis 

In [667]:
def remove_parentheses(text):
   return re.sub(r'\([^()]*\)', '', text)

# apply the function to each list element of column affilDF['uniqueAff1'] to remove substrings inside parentheses

affilDF.loc[:,'uniqueAff1'] = affilDF['uniqueAff1'].apply(lambda x: [remove_parentheses(s) for s in x])


## 4. Remove @#$%characters and umlauts

In [668]:
def replace_umlauts(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    replaced_text = ''.join(c for c in normalized_text if not unicodedata.combining(c))
    return replaced_text

affNoSymbols = []

for i in range(len(list(affilDF['uniqueAff1']))):
    L = list(affilDF['uniqueAff1'])[i]
    for j in range(len(L)):
        L[j] = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', L[j])
        L[j] = replace_umlauts(L[j])
        
    affNoSymbols.append(L)



In [669]:
affNoSymbols = [[item for item in inner_list if item != "inc"] for inner_list in affNoSymbols]


In [670]:
affilDF['uniqueAff1'] = affNoSymbols


## 5. Check 'sub'-affiliations (affiliations that are contained in other affiliations of the same DOI)

In [1128]:
newAff0 = []

for k in range(len(affilDF)):
    
    L2 = []
    for s1 in affilDF['uniqueAff1'].iloc[k]:
        is_substring = False
        for s2 in affilDF['uniqueAff1'].iloc[k]:
            if s1 != s2 and s1 in s2:
                is_substring = True
                break
        if not is_substring:
            L2.append(s1)
    newAff0.append(L2)

In [1129]:
newAffList = [list(set(newAff0[k])) for k in range(len(newAff0))]

In [1130]:
affilDF['uniqueAff2'] = newAffList

## 6. Split strings where ',' or ';' appears    | Apply .lower()

In [1658]:

def substringsDict(string):
    split_strings = [re.sub(r'^[\s.]+|[\s.]+$', '', s.strip()) for s in re.split(r'[,;]', string)]
    dict_string = {}
    index = 0

    for value in split_strings:
        if value:
            modified_value = re.sub(r'\buniversit\w*', 'universit', value, flags=re.IGNORECASE)
            dict_string[index] = modified_value.lower()
            index += 1

    return dict_string


In [1659]:
newAffkomma = []

for k in range(len(affilDF)):
    
    new_list = []
    for item in affilDF['uniqueAff2'].iloc[k]:
        new_list.append(substringsDict(item))


    newAffkomma.append(new_list)



In [1660]:
for j in range(len(newAffkomma)):
    for y in newAffkomma[j]:
        if len(y)>1:
            for i in range(len(y)-1):
                if is_contained('progr', y[i]) and is_contained('dep', y[i+1]):
                    del y[i]
                elif (is_contained('assistant', y[i]) or is_contained('researcher', y[i]) or is_contained('phd', y[i]) or is_contained('student', y[i]) or is_contained('section', y[i]) or is_contained('prof', y[i]) or is_contained('director', y[i])) and (not is_contained('school', y[i+1]) or is_contained('univ', y[i+1]) or is_contained('inst', y[i+1]) or is_contained('lab', y[i+1]) or is_contained('fac', y[i+1])):
                    del y[i]
                elif (is_contained('engineer', y[i]) or is_contained('progr', y[i]) or is_contained('unit', y[i]) or is_contained('lab', y[i]) or is_contained('dep', y[i]) or is_contained('inst', y[i]) or is_contained('hosp', y[i]) or is_contained('school', y[i]) or is_contained('fac', y[i])) and is_contained('univ', y[i+1]):
                    del y[i]
                elif is_contained('lab', y[i]) and (is_contained('college', y[i+1]) or is_contained('inst', y[i+1]) or is_contained('dep', y[i+1]) or is_contained('school', y[i+1])):
                    del y[i]
                elif is_contained('dep', y[i]) and (is_contained('college', y[i+1]) or is_contained('inst', y[i+1]) or  is_contained('hosp', y[i+1]) or  is_contained('school', y[i+1]) or  is_contained('fac', y[i+1])):
                    del y[i]
                elif is_contained('inst', y[i]) and (is_contained('dep', y[i+1]) or is_contained('acad', y[i+1]) or is_contained('hosp', y[i+1]) or is_contained('fac', y[i+1]) or is_contained('cent', y[i+1]) or is_contained('div', y[i+1])):
                    del y[i]
            #    elif y[i] in city_names+removeList:
             #       del y[i]

            

In [1661]:
lightAff = []
for j in range(len(newAffkomma)):
    lightAffj = []
    for y in newAffkomma[j]:
        lightAffj.append(', '.join(list(y.values())))
    lightAff.append(lightAffj)

In [1662]:
lightAff0 = []

for k in range(len(lightAff)):
    
    L2 = []
    for s1 in lightAff[k]:
        is_substring = False
        for s2 in lightAff[k]:
            if s1 != s2 and s1 in s2:
                is_substring = True
                break
        if not is_substring:
            L2.append(s1)
    lightAff0.append(L2)

In [1664]:
affilDF['lightAff'] = lightAff0

In [1665]:
removeList = ['university','research institute','laboratory' , 'universit','gmbh', 'inc', 'university of', 'research center', 
'university college','national institute of', 'school of medicine', "university school", 'graduate school of', 'graduate school of engineering', 
'institute of tropical medicine', 'institute of virology', 'faculty of medicine','laboratory', 'university park', 'institute of science','Polytechnic University']

city_names = ["Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown"]

city_names = [x.lower() for x in city_names]

In [1666]:
for j in range(len(newAffkomma)):
    for y in newAffkomma[j]:
        for i in list(y.keys()):

            if y[i] in city_names+removeList:
                del y[i]


In [1734]:
affilDF['uniqueAff4'] =  [[list(d.values()) for d in sublist] for sublist in newAffkomma]


# Labels based on legalnames of openAIRE's organizations

In [1735]:
uniList = ['escuela','institu', 'istituto','univ', 'college', 'center', 'centre' , 'cnrs', 'faculty','school' , 'academy' , 'école', 'hochschule' , 'ecole' ]

labList = ['lab']

hosplList = ['hospital' ,'clinic', 'hôpital']

gmbhList = ['gmbh', 'company' , 'industr', 'etaireia' , 'corporation', 'inc']

musList =  ['museum', 'library']

foundList =  ['foundation' , 'association','organization' ,'society', 'group' ]

deptList = ['district' , 'federation'  , 'government' , 'municipal' , 'county','council', 'agency']
# miistry -> out

unknownList = ['unknown']

#######   Dictionaries ##########

uniDict = {k: 'Univ/Inst' for k in uniList}   

labDict = {k: 'Laboratory' for k in labList} 

hosplDict = {k: 'Hospital' for k in hosplList}   

gmbhDict = {k: 'Company' for k in gmbhList}   

musDict = {k: 'Museum' for k in musList}   

#schoolDict = {k: 'School' for k in schoolList}   

foundDict = {k: 'Foundation' for k in foundList}   

deptDict = {k: 'Government' for k in deptList}   

unknownDict =  {k: 'Unknown' for k in unknownList}   

categDictsList = [uniDict, labDict, hosplDict, gmbhDict, musDict, #schoolDict, 
                  foundDict, deptDict, unknownDict]

################# Final Dictionary #####################

categDicts = {}
i = 0
while i in range(len(categDictsList)):
    categDicts.update(categDictsList[i])
    i = i+1
    
    


## affiliationsDict

In [1736]:
affiliationsDict = {}

for i in range(len(affilDF)):
    affiliationsDict[i] = affilDF['uniqueAff4'].iloc[i]
    


In [1737]:
d_new = {}

# iterate over the keys of affiliationsDict
for k in range(len(affiliationsDict)):
    mappedk = []
    # get the list associated with the current key in affiliationsDict
    L = affiliationsDict.get(k, [])
    
    for x in L:
        mapped_listx = [[s, v] for s in x for k2, v in categDicts.items() if k2 in s]
        mappedk.append(mapped_listx)
    
    # add the mapped list to the new dictionary d_new
    d_new[k] = mappedk

In [1738]:
affilDF['Dictionary'] = d_new

In [1739]:
notInList = [i for i in range(len(affilDF)) if affilDF['Dictionary'].iloc[i] == [[]]]
    

In [1740]:
len(affilDF)  - len(notInList)

844

In [1741]:
len(notInList)

43

# affilDF1 ['DOI', 'affiliations', 'Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']

In [1742]:
affilDF.head(10)

,items,DOI,authors,# authors,affiliations,# Affil,uniqueAff,# uniqueAff,uniqueAff1,uniqueAff2,lightAff,uniqueAff4,Dictionary
0,"{'URL': 'http://dx.doi.org/10.1021/ac020069k',...",10.1021/ac020069k,"[{'given': 'Gi Hun', 'family': 'Seong', 'seque...",3,"[[{'name': 'Department of Chemistry, Texas A&M...",3,"[Department of Chemistry, Texas A&M University...",1,"[Department Chemistry, Texas AM University, PO...","[Department Chemistry, Texas AM University, PO...","[texas am universit, po box 30012, college sta...","[[texas am universit, po box 30012, texas 7784...","[[[texas am universit, Univ/Inst]]]"
1,{'abstract': '<jats:p> <jats:bold>...,10.1161/01.cir.0000013846.72805.7e,"[{'given': 'Dinesh K.', 'family': 'Kalra', 'se...",10,[[{'name': 'From the Section of Cardiology (D....,10,"[From the Section of Cardiology (D.K.K., X.Z.,...",1,[Section Cardiology and Echocardiography Labo...,[Section Cardiology and Echocardiography Labo...,"[baylor college medicine, houston, tex]","[[baylor college medicine, tex]]","[[[baylor college medicine, Univ/Inst]]]"
2,{'abstract': '<jats:p>Hyperhomocysteinemia (hH...,10.1161/hy02t2.102992,"[{'given': 'Ningjun', 'family': 'Li', 'sequenc...",3,"[[{'name': 'From the Department of Physiology,...",3,"[From the Department of Physiology, Medical Co...",1,"[Department Physiology, Medical College Wiscon...","[Department Physiology, Medical College Wiscon...","[medical college wisconsin, milwaukee, tongji ...","[[medical college wisconsin, tongji medical co...","[[[medical college wisconsin, Univ/Inst], [ton..."
3,{'abstract': '<jats:p> The nuclear...,10.1126/science.1073633,"[{'given': 'Jianping', 'family': 'Hu', 'sequen...",6,"[[{'name': 'Plant Biology Laboratory,'}, {'nam...",6,"[Plant Biology Laboratory,, Howard Hughes Medi...",3,"[Plant Biology Laboratory,, Howard Hughes Medi...","[Plant Biology Laboratory,, Howard Hughes Medi...","[plant biology laboratory, howard hughes medic...","[[plant biology laboratory], [howard hughes me...","[[[plant biology laboratory, Laboratory]], [[h..."
4,{'URL': 'http://dx.doi.org/10.1089/10872910260...,10.1089/10872910260066679,"[{'given': 'Edward E.', 'family': 'Eaton', 'se...",5,[[{'name': 'Albert Einstein College of Medicin...,5,"[Einstein College of Medicine, Bronx, New York...",4,"[Einstein College Medicine, Bronx, New York an...","[Albert Einstein College Medicine, Bronx, New ...","[albert einstein college medicine, bronx, new ...","[[albert einstein college medicine, bronx, new...","[[[albert einstein college medicine, Univ/Inst..."
5,{'URL': 'http://dx.doi.org/10.1126/science.296...,10.1126/science.296.5576.2123a,"[{'given': 'Wayne', 'family': 'Kondro', 'seque...",1,[[{'name': 'Wayne Kondro writes from Ottawa.'}]],1,[Wayne Kondro writes from Ottawa.],1,[Wayne Kondro writes Ottawa],[Wayne Kondro writes Ottawa],[wayne kondro writes ottawa],[[wayne kondro writes ottawa]],[[]]
6,{'URL': 'http://dx.doi.org/10.1046/j.1440-1665...,10.1046/j.1440-1665.2002.t01-1-00428.x,"[{'given': 'Bob', 'family': 'Adler', 'sequence...",1,"[[{'name': 'Melbourne, Victoria'}]]",1,"[Melbourne, Victoria]",1,"[Melbourne, Victoria]","[Melbourne, Victoria]","[melbourne, victoria]",[[victoria]],[[]]
7,{'abstract': '<jats:p>Nonresonant manipulation...,10.1126/science.1072245,"[{'given': 'Thilo M.', 'family': 'Brill', 'seq...",8,"[[{'name': 'Schlumberger-Doll Research, 36 Old...",8,"[Schlumberger-Doll Research, 36 Old Quarry Roa...",1,"[SchlumbergerDoll Research, 36 Old Quarry Road...","[SchlumbergerDoll Research, 36 Old Quarry Road...","[schlumbergerdoll research, 36 old quarry road...","[[schlumbergerdoll research, 36 old quarry roa...",[[]]
8,{'URL': 'http://dx.doi.org/10.1108/02656719610...,10.1108/02656719610116117,"[{'given': 'A.J.G.', 'family': 'Babu', 'sequen...",2,"[[{'name': 'Seton Hall University, South Orang...",2,"[Seton Hall University, South Orange, New Jers...",2,"[Seton Hall University, South Orange, New Jers...","[Seton Hall University, South Orange, New Jers...","[seton hall uni

In [1743]:
affilDF1 = affilDF[['DOI', 'affiliations','lightAff','Dictionary','uniqueAff4', 'uniqueAff2','# authors','# uniqueAff']]

In [1744]:
len(affilDF1)

887

In [1745]:
len(affilDF1.drop_duplicates(subset=['DOI']))

887

## New column: category based on the labels 

In [1746]:
category = [', '.join(list(set([x[1] for y in affilDF1['Dictionary'].iloc[i] for x in y]))) for i in range(len(affilDF1))]
    


In [1747]:
affilDF1 = affilDF1.copy()

In [1748]:
affilDF1.loc[:, 'category'] = category


### new label: rest

In [1749]:
for i in range(len(affilDF1)):
    if affilDF1['category'].iloc[i] == '':
        affilDF1.iloc[i, affilDF1.columns.get_loc('category')] = 'Rest'


In [1750]:
affiliationsSimple = [
    list(set([inner_list[0] for outer_list in affilDF1['Dictionary'].iloc[i] for inner_list in outer_list]))
    for i in range(len(affilDF1))
]


In [1751]:
affilDF1.loc[:,'affilSimple'] = affiliationsSimple

In [1752]:
def strRadiusU(string):
    string = string.lower()
    radius = 3
    
    strList = string.split()
    indices = []
    result = []

    for i, x in enumerate(strList):
        if is_contained('univers',x):
            indices.append(i)
            
    for r0 in indices:
        lmin =max(0,r0-radius)
        lmax =min(r0+radius, len(strList))
        s = strList[lmin:lmax]
        
        result.append(' '.join(s))
    
    return result 

In [1753]:
def strRadiusH(string):
    string = string.lower()
    radius = 3
    
    strList = string.split()
    indices = []
    result = []

    for i, x in enumerate(strList):
        if is_contained('hospital',x):
            indices.append(i)
            
    for r0 in indices:
        lmin =max(0,r0-radius-1)
        lmax =min(r0+radius, len(strList))
        s = strList[lmin:lmax]
        
        result.append(' '.join(s))
    
    return result 

In [1754]:
affiliationsSimpleN = []

for i in range(len(affiliationsSimple)):
    inner = []
    for str in affiliationsSimple[i]:
        if 'university' in str.split():
            for x in strRadiusU(str):
                inner.append(x)
        elif 'hospital' in str.split():
            for x in strRadiusH(str):
                inner.append(x)
        else:
            inner.append(str)
    
    affiliationsSimpleN.append(inner)      
            
            
    

In [1690]:
affilDF1.loc[:,'affilSimpleNew'] = affiliationsSimpleN

In [1691]:
len(affilDF1[affilDF1['category'] == 'Rest'])

46

# UNIVS & LABS

In [1692]:
univLabs = [i for i in range(len(affilDF1)) if 'Laboratory' in affilDF1['category'].iloc[i] 
            or 'Univ/Inst' in  affilDF1['category'].iloc[i]]

In [1693]:

univLabsDF = affilDF1.iloc[univLabs].copy()

In [1694]:
univLabsDF.reset_index(inplace = True)

In [1695]:
univLabsDF.drop(columns = ['index'], inplace = True)

# Load files from openAIRE

In [1696]:
#with open('dixOpenAIRE_Alletc.pkl', 'rb') as f:
#    dixOpenAIRE_Alletc = pickle.load(f)

#with open('dixOpenAIRE_id.pkl', 'rb') as f:
#    dixOpenAIRE_id = pickle.load(f)


In [1697]:
with open('dixOpenOrgId.pkl', 'rb') as f:
    dixOpenOrgId = pickle.load(f)


## Clean/modify the files

In [1698]:
#dixOpenAIRE_Alletc1 =  {k.replace(',', ''): v for k, v in dixOpenAIRE_Alletc.items()}
#dixOpenAIRE_id1 = {k.replace(',', ''): v for k, v in dixOpenAIRE_id.items()}

In [1699]:
dixOpenOrgId1 = {k.replace(',', ''): v for k, v in dixOpenOrgId.items()}

In [1700]:
dixOpenOrgId1 = {
    replace_umlauts(key): value
    for key, value in dixOpenOrgId1.items()
}


In [1701]:
def filter_key(key):
    # Remove all non-alphanumeric characters except Greek letters and Chinese characters
    modified_key = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', key)
    modified_key = re.sub(r'\buniversit\w*', 'universit', modified_key, flags=re.IGNORECASE)
    return modified_key

    
def filter_dictionary_keys(dictionary):
    filtered_dict = {}
    for key, value in dictionary.items():
        filtered_key = filter_key(key)
        filtered_dict[filtered_key] = value
    return filtered_dict


#dixOpenAIRE_Alletc1 = filter_dictionary_keys(dixOpenAIRE_Alletc1)
#dixOpenAIRE_id1 = filter_dictionary_keys(dixOpenAIRE_id1)
dixOpenOrgId1 = filter_dictionary_keys(dixOpenOrgId1)



In [1702]:

dixOpenOrgId2 = {}
for key, value in dixOpenOrgId1.items():
    updated_key = ' '.join([word for word in key.split() if word.lower() not in ['of', 'at', "the", 'de']])
    dixOpenOrgId2[updated_key] = value


In [1703]:
#del dixOpenAIRE_Alletc1['laboratory']
#del dixOpenAIRE_Alletc1['university hospital']

In [1704]:
#del dixOpenAIRE_id1['laboratory']
#del dixOpenAIRE_id1['university hospital']

In [1705]:
del dixOpenOrgId2['universit hospital']

In [1706]:
del dixOpenOrgId2['universit school']

In [1707]:
del dixOpenOrgId2['us']

In [1708]:
del dixOpenOrgId2['ni universit']
del dixOpenOrgId2['s v universit']

In [1709]:
del dixOpenOrgId2['k l universit']


In [1710]:
def findID(name):
    lnames = []
    for x in list(dixOpenOrgId2.keys()):
        if name.lower() in x:
            lnames.append(x)
    return lnames

# MATCHINGS

## Helper functions

### Clean the matchings

In [1756]:
def bestSimScore(l1, l2, l3, l4, simU, simG):
    """
    Finds the best match between a 'key word' and several legal names from the OpenAIRE database.
    ---> corrects special cases in the main map that follows

    Args:
        l1: List of light affiliations.
        l2: number of candidates.
        l3: List of pairs.
        l4: mult

    Returns:
        List: Resulting list containing OpenAIRE names and their similarity scores.
    """
    
    vectorizer = CountVectorizer()
    numUniv = sum([(l1[i].lower()).count('univ') for i in range(len(l1))])
    result = []
    for i in range(len(l1)):
        best = [] 
        s = l1[i]
       # s_contains_university = is_contained("university", s.lower())  
        
       # if not is_contained("univ", s.lower()):
       #     continue  # Skip if s does not contain "university" or "univ"
        
    
        for j in range(len(l3)):
            x = l3[j][1] 
           
            if [x, l3[j][2]] in result:
                    continue
            
            if l4[l3[j][0]] == 1:
               
                if  is_contained('univ', x.lower()) and  l3[j][2]> simU:
                    result.append([x, l3[j][2]])
                elif  l3[j][2] >simG:
                    result.append([x, l3[j][2]])

                
              
            elif l3[j][2] >=0.99 and (is_contained("univ", x.lower()) or is_contained("college", x.lower()) or  is_contained("center", x.lower()) or  is_contained("schule", x.lower())): # If the similarity score of a pair (s,x) was 1, we store it to results list
                result.append([l3[j][1], 1])
                
            else:
        #        x_contains_university = is_contained("university", x.lower())
                if not is_contained("univ", x.lower()):
                    continue  # Skip if x does not contain "university" or "univ"
                
                if (is_contained('hosp', x.lower()) and not is_contained('hosp', s)) or (not is_contained('hosp', x.lower()) and is_contained('hosp', s)):
                    continue
                s_vector = vectorizer.fit_transform([s]).toarray() #Else we compute the similarity of s with the original affiiation name
                x_vector = vectorizer.transform([x]).toarray()
              #  s_vector1 =  vectorizer.transform([s]).toarray()
              #  x_vector1 =  vectorizer.fit_transform([s]).toarray()
                

                # Compute similarity between the vectors
                similarity = cosine_similarity(x_vector, s_vector)[0][0]
                if similarity> 0.31:
               # similarity1 = cosine_similarity(x_vector1, s_vector1)[0][0]
                #similarity2 = Levenshtein.ratio(s,x)


                    best.append([x, similarity])#(similarity+similarity2)/2])
        
        if best:
            max_numbers = defaultdict(float)
            for item in best:
                string, number = item
                max_numbers[string] = max(max_numbers[string], number)

# Create a new list with the elements having the maximum number for each string
            reduced_best = [[string, number] for string, number in best if number == max_numbers[string]]

#            max_score = max(best, key=lambda x: x[1])[1]
#            max_results = [(x[0], x[1]) for x in best if x[1] == max_score]
           # if len(reduced_best) > 1:
            reduced_best.sort(key=lambda x: x[1], reverse=True)
            #reduced_best.sort(key=lambda x: (l2.index(x[0]), -x[1]), reverse=False)
           #     result.append(reduced_best[-1])
            #else:
            result = result + reduced_best
                
    univ_list = []
    other_list = []
    
    for r in result:
        if is_contained('univ',r[0]):
            univ_list.append(r)
        else:
            other_list.append(r)
    
    limit =  min(numUniv, l2)

    if len(univ_list)> limit:
        result = univ_list[:limit] + other_list
                
    return result

### Find rows with multiple mathcings

In [1757]:
def index_multipleMatchings(df):
    multipleMatchings = []
    mult = []

    for i in range(len(df)):
        result_dict = {}
        

        for t in [t[0] for t in df.Pairs.iloc[i]]:
            key = t
            if key in result_dict:
                result_dict[key] += 1
                multipleMatchings.append(i)
                
            else:
                result_dict[key] = 1
        mult.append(result_dict)
    return [list(set(multipleMatchings)), mult]
                
        

## Main map

In [1758]:
def Doi_Ids(m, DF, dixOpenAIRE, simU, simG):
    
    """
    Matches affiliations in DataFrame 'DF' with names from dictionary 'dixOpenAIRE' and their openAIRE ids based on similarity scores.

    Args:
        m (int): The number of DOIs to check.
        DF (DataFrame): The input DataFrame containing affiliation data.
        dixOpenAIRE (dict): A dictionary of names from OpenAIRE.
        simU (float): Similarity threshold for universities.
        simG (float): Similarity threshold for non-universities.

    Returns:
        DataFrame: The final DataFrame with matched affiliations and their corresponding similarity scores.
    """
    
    lnamelist = list(dixOpenAIRE.keys())
    dix = {}    # will store indeces and legalnames of organizations of the DOI { i : [legalname1, legalname2,...]}
    deiktes = []  # stores indeces where a match is found
    vectorizer = CountVectorizer()
    similarity_ab = [] # stores lists of similarity scores of the mathces 
    pairs = [] #  pairs[i] =  [ [s,x,t] ] where (s,x) is a match and t the corresponding similarity score
    
    for k in range(m):
        similar_k = []
        pairs_k = []


        for s in DF['affilSimpleNew'].iloc[k]:

            if s in lnamelist:
                deiktes.append(k)
                similarity = 1
                similar_k.append(similarity)
                
                pairs_k.append((s,s,similarity))

                if k not in dix:
                    dix[k] = [s]
                else:
                    dix[k].append(s)
            else:

                for x in lnamelist:
                    
                    if  is_contained(s, x):
                        x_vector = vectorizer.fit_transform([x]).toarray()
                        s_vector = vectorizer.transform([s]).toarray()

                        # Compute similarity between the vectors
                        similarity = cosine_similarity(x_vector, s_vector)[0][0]
                        if similarity > min(simU, simG):
                            if (is_contained('univ', s) and is_contained('univ', x)) and similarity > simU:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))

                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                            elif (not is_contained('univ', s) and not is_contained('univ', x)) and similarity > simG:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))

                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                    elif is_contained(x, s):
                        if (is_contained('univ', s) and is_contained('univ', x)):

                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t) and is_contained('univ', t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simU:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))

                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > simU: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))

                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                        elif not is_contained('univ', s) and not is_contained('univ', x):
                            if 'and' in s:
                                list_s = s.split(' and ')
                                for t in list_s:
                                    if is_contained(x, t):
                                        t_vector = vectorizer.fit_transform([t]).toarray()
                                        x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                                        similarity = cosine_similarity(t_vector, x_vector)[0][0]
                                        if similarity > simG:
                                            similar_k.append(similarity)
                                            deiktes.append(k)
                                            pairs_k.append((s,x,similarity))

                                            if k not in dix:
                                                dix[k] = [x]
                                            else:
                                                dix[k].append(x)
                            
                            else: 
                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > simG: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))

                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                            
        similarity_ab.append(similar_k)   
        similarity_ab = [lst for lst in similarity_ab if lst != []]
        pairs.append(pairs_k)
        
    
    dixDoiAff = {DF['DOI'].iloc[key]: value for key, value in dix.items()} # dictionary {DOI : legalnames} 
    
    #dixDoiPid1 = {key : [dixOpenAIRE[x] for x in value if x in  lnamelist] for key , value in dixDoiAff.items()}
    
   # dixDoiPid = {key : [dixOpenAIRE[x] for x in value] for key , value in dixDoiAff.items()} # dictionary {DOI : PIDs} 
    
    
    
## Define the new Dataframe
    
    doiIdDF = pd.DataFrame()
    doiIdDF['DOI'] = list(dixDoiAff.keys())
    doiIdDF['Affiliations'] = list(DF['affiliations'].iloc[list(set(deiktes))])

    doiIdDF['Unique affiliations'] = list(DF['uniqueAff2'].iloc[list(set(deiktes))])
    doiIdDF['light affiliations'] = list(DF['lightAff'].iloc[list(set(deiktes))])

    
    doiIdDF['# Authors'] = list(DF['# authors'].iloc[list(set(deiktes))])


    doiIdDF['# Unique affiliations'] = list(DF['# uniqueAff'].iloc[list(set(deiktes))])

    doiIdDF['Candidates for matching'] = list(DF['affilSimpleNew'].iloc[list(set(deiktes))])
    doiIdDF['Candidates old'] = list(DF['affilSimple'].iloc[list(set(deiktes))])


    doiIdDF['Matched openAIRE names'] = list(dix.values())
    doiIdDF['# Matched orgs'] = [len(list(dix.values())[i]) for i in range(len(list(dix.values())))]
    

    doiIdDF['Similarity score'] = similarity_ab
    #perfectSim = [[1 if num >= 1 else 0 for num in inner_list] for inner_list in similarity_ab]

    #doiIdDF['Perfect match'] = perfectSim
    #perfectSimSum = [sum(x) for x in perfectSim]
    #doiIdDF['Perfect sum'] = perfectSimSum
    Pairs = [lst for lst in pairs if lst]
    doiIdDF['Pairs'] = Pairs
    doiIdDF['mult'] = index_multipleMatchings(doiIdDF)[1]
    #doiIdDF['Cleaning'] =  list(DF['Cleaning'].iloc[list(set(deiktes))])


    


## Correct the matchings
    needCheck = list(set([i for i in range(len(doiIdDF)) for k in list(doiIdDF['mult'].iloc[i].values()) if k>1]))
    
    #needCheck = [i for i  in range(len(doiIdDF)) if doiIdDF['# Matched orgs'].iloc[i] - max(doiIdDF['# Authors'].iloc[i],doiIdDF['# Unique affiliations'].iloc[i]) >0 or    i in index_multipleMatchings(doiIdDF)[0]]

    ready = [i for i in range(len(doiIdDF)) if i not in needCheck]
   
    best = [ bestSimScore1(doiIdDF['light affiliations'].iloc[i], len(doiIdDF['Candidates for matching'].iloc[i]), doiIdDF['Pairs'].iloc[i],doiIdDF['mult'].iloc[i], simU, simG) for i in needCheck]
    #best = [ bestSimScore(doiIdDF['light affiliations'].iloc[i], doiIdDF['Matched openAIRE names'].iloc[i], doiIdDF['Pairs'].iloc[i]) for i in needCheck]
    best_o = []
    best_s = []
    
    for x in best:
        best_o.append([x[i][0]  for i in range(len(x))])
        best_s.append([round(x[i][1],2)  for i in range(len(x))])
    numMathced = [len(best_s[i]) for i in range(len(needCheck))]
    

    
    dfFinal0 = (doiIdDF.iloc[ready]).copy()
    dfFinal0['index'] = ready
    
    dfFinal1 = (doiIdDF.iloc[needCheck]).copy()
    dfFinal1['index'] = needCheck
    dfFinal1['Matched openAIRE names'] = best_o
    dfFinal1['Similarity score'] = best_s
    dfFinal1['# Matched orgs'] = numMathced
    
    finalDF =  pd.concat([dfFinal0, dfFinal1])
    finalDF.set_index('index', inplace=True)
    finalDF.sort_values('index', ascending=True, inplace = True)
    
    ids = [[dixOpenAIRE[x] for x in v] for v in finalDF['Matched openAIRE names']]
    numIds = [len(x) for x in ids]

    finalDF['IDs'] = ids
    finalDF['# IDs'] = numIds
    finalDF = finalDF[~(finalDF['# Matched orgs'] == 0)]
    
    finalDF = finalDF.reset_index(drop=True)
    perc = 100*len(finalDF)/m



    
    
    return [perc,finalDF,doiIdDF, needCheck]
    


In [1759]:
result = Doi_Ids(len(univLabsDF), univLabsDF, dixOpenOrgId2, 0.7,0.82)

In [1760]:
result[0]

88.34355828220859

# HTML

In [1764]:
finaldf = result[1]
finalDF = finaldf[["Affiliations", "Unique affiliations",'# Authors','# Unique affiliations', '# Matched orgs', 'Matched openAIRE names', 'Similarity score']]


In [1762]:
def update_Z(row):
    new_Z = []
    for i in range(len(row['IDs'])):
        entry = {'openaireId': row['IDs'][i], 'confidence': row['Similarity score'][i]}
        new_Z.append(entry)
    return new_Z

# Update the values in column 'Z' using 'apply'
finaldf['affiliations'] = finaldf.apply(update_Z, axis=1)


In [1763]:
finaldf_output = finaldf[['DOI','affiliations']]
finaldf_output = finaldf_output.rename(columns={'DOI': 'doi'}).copy()



In [1766]:
app = dp.App(#dp.Page(title="Matchings [explained]", blocks= [dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.DataTable(finalDF.drop(columns =['Candidates old']))]
             #), 
             dp.Page(title="Matchings [final]", 
                     blocks= [dp.Text('DOIs with aff. in Univ/Inst, Laboratories'), dp.Table(finalDF)]
             )
             )
    


   
app.save(path="test.html", open=True)


App saved to ./test.html

# JSON

In [1731]:
match0 = finaldf_output.to_json(orient='records', lines=True)

# Save the JSON to a file
with open('match0.json', 'w') as f:
    f.write(match0)

# proxeiro

In [1732]:
findID('provence')

['centre provencealpescote dazur',
 'institut des materiaux microelectronique et des nanosciences provence',
 'universit provence']

In [1733]:
vectorizer = CountVectorizer()


s = 'university provence'
x = 'universite provence'

s_vector = vectorizer.fit_transform([s]).toarray()
x_vector = vectorizer.transform([x]).toarray()

# Compute similarity between the vectors
similarity = cosine_similarity(s_vector, x_vector)[0][0]
similarity

0.7071067811865475